In [1]:
# Cài đặt thư viện (nếu chưa có)
# %pip install pandas numpy pyarrow

import pandas as pd
import numpy as np
import warnings

# Tắt các cảnh báo đỏ không cần thiết
warnings.filterwarnings('ignore')

print("✅ Đã khởi động xong thư viện!")

✅ Đã khởi động xong thư viện!


In [2]:
def load_data_optimized(file_path):
    print(f"🔄 Đang đọc file '{file_path}'...")
    
    # KỸ THUẬT TĂNG TỐC: Định nghĩa trước kiểu dữ liệu
    column_types = {
        'manufacturer': 'string',
        'model': 'string',
        'year': 'int32',
        'mileage': 'float32',
        'price': 'float32',
        'mpg': 'string', # MPG giữ là chuỗi vì có dạng '38-39'
        'accidents_or_damage': 'float32'
    }
    
    cols_to_use = list(column_types.keys())

    try:
        # Thử dùng engine 'pyarrow' (nhanh nhất)
        df = pd.read_csv(file_path, usecols=cols_to_use, dtype=column_types, engine='pyarrow')
    except (ImportError, ValueError):
        # Nếu máy chưa cài pyarrow, dùng engine mặc định (vẫn nhanh nhờ dtype)
        print("⚠️ Không tìm thấy engine PyArrow, chuyển sang chế độ đọc tiêu chuẩn...")
        df = pd.read_csv(file_path, usecols=cols_to_use, dtype=column_types)
    except FileNotFoundError:
        print("❌ Lỗi: Không tìm thấy file csv. Đang tạo dữ liệu mẫu để chạy thử...")
        # Dữ liệu giả lập (Backup)
        return pd.DataFrame({
            'manufacturer': ['Toyota', 'Ford', 'Honda', 'BMW', 'Lexus'],
            'model': ['Camry', 'F-150', 'Civic', '320i', 'RX350'],
            'year': [2015, 2018, 2016, 2012, 2020],
            'mileage': [50000, 30000, 80000, 100000, 15000],
            'price': [15000, 25000, 12000, 9000, 45000],
            'mpg': ['30-35', '18-24', '32-40', '25-30', '20-25'],
            'accidents_or_damage': [0, 0, 1, 0, 0]
        })

    print(f"✅ Đã tải thành công {len(df):,} dòng dữ liệu.")
    return df

# --- TẢI DỮ LIỆU ---
# Hãy thay 'cars.csv' bằng tên file thật của bạn
df_raw = load_data_optimized('cars.csv')

🔄 Đang đọc file 'cars.csv'...
✅ Đã tải thành công 762,091 dòng dữ liệu.


In [3]:
# =======================================================
# BẢNG ĐÁNH GIÁ ĐỘ QUAN TRỌNG (NGƯỜI DÙNG SỬA TẠI ĐÂY)
# =======================================================

# 1. So sánh GIÁ (Price) với các tiêu chí khác:
PRICE_vs_YEAR      = 3      # Giá quan trọng gấp 3 lần Năm sx
PRICE_vs_MILEAGE   = 2      # Giá quan trọng gấp 2 lần Số KM đã đi
PRICE_vs_MPG       = 4      # Giá quan trọng gấp 4 lần Mức tiêu hao xăng
PRICE_vs_ACCIDENTS = 1      # Giá quan trọng NGANG BẰNG Lịch sử tai nạn

# 2. So sánh NĂM (Year) với các tiêu chí còn lại:
YEAR_vs_MILEAGE    = 1/2    # Năm sx KHÔNG quan trọng bằng Số KM (Số KM quan trọng gấp đôi)
YEAR_vs_MPG        = 2      # Năm sx quan trọng gấp 2 lần Xăng
YEAR_vs_ACCIDENTS  = 1/3    # Năm sx KHÔNG quan trọng bằng Tai nạn

# 3. So sánh SỐ KM (Mileage) với các tiêu chí còn lại:
MILEAGE_vs_MPG       = 3    # Số KM quan trọng gấp 3 lần Xăng
MILEAGE_vs_ACCIDENTS = 1/2  # Số KM KHÔNG quan trọng bằng Tai nạn

# 4. So sánh XĂNG (MPG) với Tai nạn:
MPG_vs_ACCIDENTS     = 1/5  # Xăng KHÔNG quan trọng bằng Tai nạn (Tai nạn quan trọng gấp 5)

print("✅ Đã lưu cấu hình đánh giá của bạn!")

✅ Đã lưu cấu hình đánh giá của bạn!


In [4]:
def calculate_ahp_weights():
    criteria = ['Price', 'Year', 'Mileage', 'MPG', 'Accidents']
    n = len(criteria)
    matrix = np.ones((n, n))
    
    # Điền giá trị vào ma trận từ cấu hình người dùng
    # Hàng Price
    matrix[0, 1] = PRICE_vs_YEAR;      matrix[1, 0] = 1/PRICE_vs_YEAR
    matrix[0, 2] = PRICE_vs_MILEAGE;   matrix[2, 0] = 1/PRICE_vs_MILEAGE
    matrix[0, 3] = PRICE_vs_MPG;       matrix[3, 0] = 1/PRICE_vs_MPG
    matrix[0, 4] = PRICE_vs_ACCIDENTS; matrix[4, 0] = 1/PRICE_vs_ACCIDENTS
    
    # Hàng Year
    matrix[1, 2] = YEAR_vs_MILEAGE;    matrix[2, 1] = 1/YEAR_vs_MILEAGE
    matrix[1, 3] = YEAR_vs_MPG;        matrix[3, 1] = 1/YEAR_vs_MPG
    matrix[1, 4] = YEAR_vs_ACCIDENTS;  matrix[4, 1] = 1/YEAR_vs_ACCIDENTS
    
    # Hàng Mileage
    matrix[2, 3] = MILEAGE_vs_MPG;       matrix[3, 2] = 1/MILEAGE_vs_MPG
    matrix[2, 4] = MILEAGE_vs_ACCIDENTS; matrix[4, 2] = 1/MILEAGE_vs_ACCIDENTS
    
    # Hàng MPG
    matrix[3, 4] = MPG_vs_ACCIDENTS;     matrix[4, 3] = 1/MPG_vs_ACCIDENTS

    # Tính toán vector trọng số (Eigenvector method approximation)
    col_sums = matrix.sum(axis=0)
    normalized_matrix = matrix / col_sums
    weights = normalized_matrix.mean(axis=1)
    
    return criteria, weights

# Chạy tính toán và hiển thị
criteria_names, weights = calculate_ahp_weights()

print("\n📊 BẢNG TRỌNG SỐ ĐÃ TÍNH ĐƯỢC:")
print("-" * 40)
for name, w in zip(criteria_names, weights):
    print(f"• {name:<10} : {w*100:6.2f}%")
print("-" * 40)


📊 BẢNG TRỌNG SỐ ĐÃ TÍNH ĐƯỢC:
----------------------------------------
• Price      :  31.59%
• Year       :  10.85%
• Mileage    :  18.18%
• MPG        :   6.47%
• Accidents  :  32.92%
----------------------------------------


In [5]:
def process_ranking(df, weights):
    # 1. Hàm xử lý cột MPG (chuyển chuỗi '30-40' thành số trung bình)
    def parse_mpg(val):
        if pd.isna(val): return 0
        s = str(val)
        if '-' in s:
            try:
                parts = s.split('-')
                return (float(parts[0]) + float(parts[1])) / 2
            except: return 0
        try: return float(s)
        except: return 0

    df = df.copy()
    df['mpg_num'] = df['mpg'].apply(parse_mpg)
    
    # 2. Chuẩn hóa dữ liệu (Normalization)
    df_norm = pd.DataFrame()
    
    # -- Nhóm Cost (Càng thấp càng tốt) --
    # Giá thấp = Tốt; Tai nạn ít = Tốt; ODO thấp = Tốt
    for col in ['price', 'mileage', 'accidents_or_damage']:
        val = df[col].astype(float).fillna(0)
        if val.max() != val.min():
            df_norm[col] = (val.max() - val) / (val.max() - val.min())
        else:
            df_norm[col] = 1.0

    # -- Nhóm Benefit (Càng cao càng tốt) --
    # Năm mới = Tốt; MPG cao = Tốt
    for col_raw, col_norm in [('year', 'year'), ('mpg_num', 'mpg')]:
        val = df[col_raw].astype(float).fillna(0)
        if val.max() != val.min():
            df_norm[col_norm] = (val - val.min()) / (val.max() - val.min())
        else:
            df_norm[col_norm] = 1.0
            
    # 3. Tính điểm tổng hợp (Weighted Sum)
    df['Final_Score'] = (
        df_norm['price'] * weights[0] +
        df_norm['year'] * weights[1] +
        df_norm['mileage'] * weights[2] +
        df_norm['mpg'] * weights[3] +
        df_norm['accidents_or_damage'] * weights[4]
    )
    
    return df

print("✅ Đã nạp hàm tính điểm!")

✅ Đã nạp hàm tính điểm!


In [6]:
# Thực hiện xếp hạng
df_result = process_ranking(df_raw, weights)

# Lấy Top 10
top_cars = df_result.sort_values(by='Final_Score', ascending=False).head(10)

# Hiển thị kết quả đẹp mắt
print("\n" + "="*80)
print("🏆 TOP 10 XE ĐÁNG MUA NHẤT THEO TIÊU CHÍ CỦA BẠN")
print("="*80)

# Chọn cột để hiển thị
cols = ['manufacturer', 'model', 'year', 'price', 'mileage', 'mpg', 'accidents_or_damage', 'Final_Score']

# Định dạng hiển thị
pd.options.display.float_format = '{:,.2f}'.format
print(top_cars[cols].to_string(index=False))

# (Tùy chọn) Lưu kết quả ra file Excel nếu muốn
# top_cars.to_excel('ket_qua_top_10.xlsx', index=False)


🏆 TOP 10 XE ĐÁNG MUA NHẤT THEO TIÊU CHÍ CỦA BẠN
manufacturer                     model  year     price   mileage     mpg  accidents_or_damage  Final_Score
      Toyota                  Prius LE  2021 28,999.00  2,899.00 121-121                 0.00         0.99
        Ford              F-250 Lariat  2019 83,990.00 20,100.00   0-255                 0.00         0.99
      Toyota          Prius Prime Plus  2018 27,500.00 33,576.00 121-121                 0.00         0.99
      Toyota       Prius Prime Premium  2018 28,995.00 53,642.00 121-121                 0.00         0.98
  Volkswagen    Jetta 1.4T SEL Premium  2019 19,790.00 53,265.00 126-111                 0.00         0.98
        Audi Q4 e-tron 50 Premium Plus  2022 61,890.00    588.00  100-89                 0.00         0.98
        Audi Q4 e-tron 50 Premium Plus  2022 61,044.00  1,159.00  100-89                 0.00         0.98
        Audi Q4 e-tron 50 Premium Plus  2022 60,980.00  1,273.00  100-89                 0.00  